In [92]:
import os
from flask import Flask, request, jsonify
import requests
import json
import base64
#from cfenv import AppEnv
import pyhdb
import pulp
#import xlrd
import numpy as np
import math
from math import isnan
import json
import pandas as pd
import time
import copy

In [4]:
f = open("New Test Data.txt",encoding = "utf-8-sig")
line = f.readlines()
f.close()
responseText = line[0]

In [5]:
try:
    if responseText.startswith(u'\ufeff'):
        responseText = responseText.encode('utf8')[3:].decode('utf8') 
    data = json.loads(responseText,encoding = "UTF-8")
except:
    if responseText.startswith(u'\ufeff'):
        responseText = responseText.encode('utf8')[3:].decode('utf8') 
    data = responseText

In [6]:
def get_connection():
    conn = pyhdb.connect(host = "10.47.84.196", #HANA地址
                         port = 30015, #HANA端口号
                         user = "I322907", #用户名
                         password = "Sap12345") #密码
    return conn


def get_efficiency(conn): #有可能用worker output那张表来解决
    cursor = conn.cursor()
    cursor.execute('Select "WORKER_ID", "PROJECT_ID","PRODUCT_ID","OUTPUT_DATE", "OUTPUT" from ( Select A.*,row_number()over(partition by A.WORKER_ID,A.PRODUCT_ID order by  A.OUTPUT_DATE DESC) as RID from "IFACTORY"."IFactory.Data::table.WORKER_OUTPUT_V" A    where A."WORKER_ID" in (select "WORKER_ID" from "IFACTORY"."IFactory.Data::table.WORKER_DETAIL")   ) where RID < \'21\' and "OUTPUT"> \'0\'',{})
    mat = cursor.fetchall()
    mat = pd.DataFrame(mat)#从数据库中存储efficiency，做成df是为了方便管理
    try:
        cursor.close()
    except:
        pass
    return mat


def get_order(conn):
    cursor = conn.cursor()
    cursor.execute('SELECT * FROM "IFACTORY"."IFactory.Data::table.ORDER_PLAN_TEMP_V";',{})
    mat = cursor.fetchall()
    mat = pd.DataFrame(mat)#存储order信息，同上
    try:
        cursor.close()
    except:
        pass
    return mat


def get_procedure(conn):
    cursor = conn.cursor()
    cursor.execute('SELECT * FROM "IFACTORY"."IFactory.Data::table.PROJECT_PRODUCT_NV";',{})
    mat = cursor.fetchall()
    mat = pd.DataFrame(mat)#存储order信息，同上
    try:
        cursor.close()
    except:
        pass
    return mat

In [7]:
conn = get_connection()
efficiency = get_efficiency(conn)

In [115]:
data

{'output': [{'WORKER_ID': '100002',
   'PROJECT_ID': '400003',
   'PRODUCT_ID': '500004',
   'OUTPUT_DATE': '0000-00-00',
   'OUTPUT': '400'},
  {'WORKER_ID': '100002',
   'PROJECT_ID': '400003',
   'PRODUCT_ID': '500005',
   'OUTPUT_DATE': '0000-00-00',
   'OUTPUT': '74.25'},
  {'WORKER_ID': '100002',
   'PROJECT_ID': '400004',
   'PRODUCT_ID': '500006',
   'OUTPUT_DATE': '0000-00-00',
   'OUTPUT': '508.27'},
  {'WORKER_ID': '100002',
   'PROJECT_ID': '400004',
   'PRODUCT_ID': '500007',
   'OUTPUT_DATE': '0000-00-00',
   'OUTPUT': '375.68'},
  {'WORKER_ID': '100002',
   'PROJECT_ID': '400006',
   'PRODUCT_ID': '500012',
   'OUTPUT_DATE': '0000-00-00',
   'OUTPUT': '324.59'},
  {'WORKER_ID': '100002',
   'PROJECT_ID': '400007',
   'PRODUCT_ID': '500016',
   'OUTPUT_DATE': '0000-00-00',
   'OUTPUT': '264.4'},
  {'WORKER_ID': '100002',
   'PROJECT_ID': '400008',
   'PRODUCT_ID': '500017',
   'OUTPUT_DATE': '0000-00-00',
   'OUTPUT': '365.08'},
  {'WORKER_ID': '100002',
   'PROJECT_ID': 

In [116]:
# efficiency = pd.DataFrame(data["output"])

In [97]:
efficiency

,0,1,2,3,4
0,100002,400003,500004,0000-00-00,400
1,100002,400003,500005,0000-00-00,74.25
2,100002,400004,500006,0000-00-00,508.27
3,100002,400004,500007,0000-00-00,375.68
4,100002,400006,500012,0000-00-00,324.59
...,...,...,...,...,...
358,100025,400012,500024,0000-00-00,180
359,100025,400013,500026,0000-00-00,140.1
360,100025,400017,500036,0000-00-00,400
361,100025,400007,500037,0000-00-00,400


In [118]:
# order = pd.DataFrame(data["plan"])

In [8]:
eff = pd.DataFrame(efficiency,index = list(map(int,np.linspace(0,len(efficiency),len(efficiency),endpoint = False).tolist())))

In [9]:
eff

,0,1,2,3,4
0,100002,400003,500004,0000-00-00,400
1,100002,400003,500005,0000-00-00,74.25
2,100002,400004,500006,0000-00-00,508.27
3,100002,400004,500007,0000-00-00,375.68
4,100002,400006,500012,0000-00-00,324.59
...,...,...,...,...,...
358,100025,400012,500024,0000-00-00,180
359,100025,400013,500026,0000-00-00,140.1
360,100025,400017,500036,0000-00-00,400
361,100025,400007,500037,0000-00-00,400


In [120]:
#eff.columns = ["output","date","product_id","project_id","worker_id"]

In [10]:
eff.columns = ['worker_id', 'project_id', 'product_id', 'date', 'output']

In [11]:
eff

,worker_id,project_id,product_id,date,output
0,100002,400003,500004,0000-00-00,400
1,100002,400003,500005,0000-00-00,74.25
2,100002,400004,500006,0000-00-00,508.27
3,100002,400004,500007,0000-00-00,375.68
4,100002,400006,500012,0000-00-00,324.59
...,...,...,...,...,...
358,100025,400012,500024,0000-00-00,180
359,100025,400013,500026,0000-00-00,140.1
360,100025,400017,500036,0000-00-00,400
361,100025,400007,500037,0000-00-00,400


In [12]:
order = get_order(conn)

In [13]:
order

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,800065,400004,舍弗勒,500006,舍弗勒塑料球子,16000,2019-09-22,0,12800,13,2019-09-05 17:53:14,2019-09-05 17:53:14,YES
1,800059,400005,安全阀,500010,检查,24000,2020-02-04,3725,16220,148,2019-09-05 17:09:36,2019-09-05 18:07:50,NO
2,800058,400006,金属片,500013,手动成形,39000,2019-12-16,0,31200,98,2019-08-20 18:53:54,2019-09-06 10:47:07,NO
3,800065,400004,舍弗勒,500007,舍弗勒金属球子,16000,2019-09-22,2000,11200,13,2019-09-05 17:53:14,2019-09-05 17:53:14,YES
4,800058,400006,金属片,500011,剪切金属片,39000,2019-12-16,5000,27200,98,2019-08-20 18:53:54,2019-09-06 10:47:07,NO
5,800059,400005,安全阀,500008,预放组件,24000,2020-02-04,4125,15900,148,2019-09-05 17:09:36,2019-09-05 18:07:50,NO
6,800058,400006,金属片,500012,压机成形,39000,2019-12-16,4887,27290,98,2019-08-20 18:53:54,2019-09-06 10:47:07,NO
7,800059,400005,安全阀,500009,推安全阀,24000,2020-02-04,4225,15820,148,2019-09-05 17:09:36,2019-09-05 18:07:50,NO


In [14]:
order.index = list(map(int,np.linspace(0,len(order),len(order),endpoint = False).tolist()))

In [15]:
if (len(order.columns) == 12):
        #order.columns = ["change_date","create_date","days","delivery_date","delivery_quan","order_id","product_id","product_name","project_id","project_name","quantity","total_output"]
        order.columns = ['order_id', 'project_id', 'project_name', 'product_id', 'product_name', 'delivery_quantity', 'delivery_date', 'total_output', 'quantity', 'days', 'create_date', 'change_date']
elif (len(order.columns) == 13):
        #order.columns = ["change_date","checked","create_date","days","delivery_date","delivery_quan","order_id","product_id","product_name","project_id","project_name","quantity","total_output"]
        order.columns = ['order_id', 'project_id', 'project_name', 'product_id', 'product_name', 'delivery_quantity', 'delivery_date', 'total_output', 'quantity', 'days', 'create_date', 'change_date', 'checked']
else:
        pass
order = order.dropna(axis = 0, how = "any") #防止出现df中有nan

In [16]:
for i in range(0,len(order["days"])):
        if int(order["days"][i]) <= 0:
            order["days"][i] = 1
        else:
            pass

In [17]:
order

,order_id,project_id,project_name,product_id,product_name,delivery_quantity,delivery_date,total_output,quantity,days,create_date,change_date,checked
0,800065,400004,舍弗勒,500006,舍弗勒塑料球子,16000,2019-09-22,0,12800,13,2019-09-05 17:53:14,2019-09-05 17:53:14,YES
1,800059,400005,安全阀,500010,检查,24000,2020-02-04,3725,16220,148,2019-09-05 17:09:36,2019-09-05 18:07:50,NO
2,800058,400006,金属片,500013,手动成形,39000,2019-12-16,0,31200,98,2019-08-20 18:53:54,2019-09-06 10:47:07,NO
3,800065,400004,舍弗勒,500007,舍弗勒金属球子,16000,2019-09-22,2000,11200,13,2019-09-05 17:53:14,2019-09-05 17:53:14,YES
4,800058,400006,金属片,500011,剪切金属片,39000,2019-12-16,5000,27200,98,2019-08-20 18:53:54,2019-09-06 10:47:07,NO
5,800059,400005,安全阀,500008,预放组件,24000,2020-02-04,4125,15900,148,2019-09-05 17:09:36,2019-09-05 18:07:50,NO
6,800058,400006,金属片,500012,压机成形,39000,2019-12-16,4887,27290,98,2019-08-20 18:53:54,2019-09-06 10:47:07,NO
7,800059,400005,安全阀,500009,推安全阀,24000,2020-02-04,4225,15820,148,2019-09-05 17:09:36,2019-09-05 18:07:50,NO


In [18]:
m_num = len(eff["worker_id"].drop_duplicates(keep = "first"))

In [19]:
m_num

24

In [20]:
n_num = len(order["product_id"].drop_duplicates(keep = "first"))

In [21]:
n_num

8

In [22]:
s_num = 4

In [23]:
worker = sorted(eff["worker_id"].drop_duplicates(keep = "first"))

In [24]:
worker

['100002',
 '100003',
 '100004',
 '100005',
 '100006',
 '100007',
 '100008',
 '100009',
 '100010',
 '100011',
 '100012',
 '100013',
 '100014',
 '100015',
 '100016',
 '100017',
 '100018',
 '100019',
 '100020',
 '100021',
 '100022',
 '100023',
 '100024',
 '100025']

In [25]:
product = sorted(order["product_id"].drop_duplicates(keep = "first"))

In [26]:
product

['500006',
 '500007',
 '500008',
 '500009',
 '500010',
 '500011',
 '500012',
 '500013']

In [27]:
worker_efficiency = np.zeros([m_num,n_num],dtype = np.float)

In [28]:
for i in range(m_num):
        for j in range(n_num):
            worker_i = eff[eff["worker_id"] == worker[i]]
            mean = np.mean(list(map(float,worker_i[worker_i["product_id"] == product[j]]["output"].tolist())))
            worker_efficiency[i][j] = mean

c:\users\i515013\appdata\local\programs\python\python36\lib\site-packages\numpy\core\fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
c:\users\i515013\appdata\local\programs\python\python36\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [29]:
for i in range(m_num):
        for j in range(n_num):
            if isnan(worker_efficiency[i,j]) == True:
                worker_efficiency[i,j] = 0
            else:
                pass

In [30]:
worker_efficiency

array([[ 508.27,  375.68,    0.  ,    0.  ,    0.  ,    0.  ,  324.59,
           0.  ],
       [ 300.  ,  411.12,  200.  ,    0.  ,    0.  ,    0.  ,  189.6 ,
         184.71],
       [ 407.48,  598.87,  300.  ,  418.87,    0.  ,    0.  ,    0.  ,
           0.  ],
       [ 464.11,  400.  ,  300.  ,    0.  ,    0.  , 1600.  ,  400.  ,
         256.  ],
       [ 400.  ,  400.  ,    0.  ,    0.  ,    0.  ,    0.  ,    0.  ,
           0.  ],
       [ 426.  ,  254.64,  200.  ,  508.8 ,    0.  ,    0.  ,  305.94,
         293.48],
       [ 364.13,  342.65,    0.  ,    0.  ,    0.  , 1600.  ,  342.  ,
         104.53],
       [ 460.62,  420.08,    0.  ,    0.  ,    0.  ,    0.  ,  340.95,
           0.  ],
       [ 500.  ,  541.29,  247.5 ,    0.  ,    0.  , 1800.  ,  600.  ,
         645.25],
       [ 500.  ,  624.15,  250.  ,    0.  ,  398.87,    0.  ,  600.  ,
         283.63],
       [ 487.1 ,  467.67,    0.  ,    0.  ,    0.  ,    0.  ,    0.  ,
           0.  ],
       [ 420.78,  382

In [31]:
qualified_worker = []

In [32]:
for i in range(m_num):
        if sum(worker_efficiency[i,:] == 0) < n_num:
            qualified_worker.append(i)

In [33]:
qualified_worker

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 22,
 23]

In [34]:
worker_new = []

In [35]:
for i in qualified_worker:
        worker_new.append(worker[i])

In [36]:
worker_new

['100002',
 '100003',
 '100004',
 '100005',
 '100006',
 '100007',
 '100008',
 '100009',
 '100010',
 '100011',
 '100012',
 '100013',
 '100014',
 '100015',
 '100016',
 '100017',
 '100018',
 '100019',
 '100020',
 '100021',
 '100022',
 '100024',
 '100025']

In [37]:
worker_efficiency_new = worker_efficiency[qualified_worker,:]

In [38]:
worker_efficiency_new

array([[ 508.27,  375.68,    0.  ,    0.  ,    0.  ,    0.  ,  324.59,
           0.  ],
       [ 300.  ,  411.12,  200.  ,    0.  ,    0.  ,    0.  ,  189.6 ,
         184.71],
       [ 407.48,  598.87,  300.  ,  418.87,    0.  ,    0.  ,    0.  ,
           0.  ],
       [ 464.11,  400.  ,  300.  ,    0.  ,    0.  , 1600.  ,  400.  ,
         256.  ],
       [ 400.  ,  400.  ,    0.  ,    0.  ,    0.  ,    0.  ,    0.  ,
           0.  ],
       [ 426.  ,  254.64,  200.  ,  508.8 ,    0.  ,    0.  ,  305.94,
         293.48],
       [ 364.13,  342.65,    0.  ,    0.  ,    0.  , 1600.  ,  342.  ,
         104.53],
       [ 460.62,  420.08,    0.  ,    0.  ,    0.  ,    0.  ,  340.95,
           0.  ],
       [ 500.  ,  541.29,  247.5 ,    0.  ,    0.  , 1800.  ,  600.  ,
         645.25],
       [ 500.  ,  624.15,  250.  ,    0.  ,  398.87,    0.  ,  600.  ,
         283.63],
       [ 487.1 ,  467.67,    0.  ,    0.  ,    0.  ,    0.  ,    0.  ,
           0.  ],
       [ 420.78,  382

In [39]:
m_num_new = len(worker_new)

In [40]:
m_num_new

23

In [41]:
# procedure = pd.DataFrame(data["pp"])

In [42]:
procedure = get_procedure(conn)

In [43]:
procedure.columns = ["product_id","project_id","step","limit"]

In [44]:
procedure

,product_id,project_id,step,limit
0,500004,400003,1,5
1,500036,400017,1,5
2,500035,400016,5,5
3,500006,400004,1,5
4,500016,400007,1,5
5,500024,400012,1,5
6,500011,400006,1,5
7,500026,400013,1,5
8,500005,400003,2,5
9,500038,400018,1,5


In [45]:
project = order["project_id"].drop_duplicates(keep = "first")

In [46]:
print(project)

0    400004
1    400005
2    400006
Name: project_id, dtype: object


In [110]:
team = pd.DataFrame(data["team"])

In [111]:
team

,TEAM_ID,MEMBERS,PRODUCT_ID,PROJECT_ID,OUTPUT
0,200001,"100004,100007,100011","500008,500009,500010",400005,1200


In [112]:
team = pd.DataFrame(team,index = list(map(int,np.linspace(0,len(team),len(team),endpoint = False).tolist())))

In [113]:
team.columns = ["team_id","member","product_id","project_id","output"]

In [120]:
team_mem = []
team_pro = []
for index, row in team.iterrows():
    row['member'] = row['member'].split(',')
    team_mem += row['member']
    row['product_id'] = row['product_id'].split(',')
    team_pro += row['product_id']

In [52]:
team_mem

['100004', '100007', '100011']

In [53]:
team_pro

['500008', '500009', '500010']

In [54]:
t_num = len(team["team_id"].drop_duplicates(keep = "first"))

In [55]:
t_num

1

In [184]:
variables = np.array([pulp.LpVariable('x%d_%d_%d'%(i,j,k), lowBound = 0, upBound = 1,cat = pulp.LpInteger) 
    for i in range(0, m_num) for j in range(0, s_num)  for k in range(0,n_num)])

In [185]:
variables

array([x0_0_0, x0_0_1, x0_0_2, x0_0_3, x0_0_4, x0_0_5, x0_0_6, x0_0_7,
       x0_0_8, x0_0_9, x0_1_0, x0_1_1, x0_1_2, x0_1_3, x0_1_4, x0_1_5,
       x0_1_6, x0_1_7, x0_1_8, x0_1_9, x0_2_0, x0_2_1, x0_2_2, x0_2_3,
       x0_2_4, x0_2_5, x0_2_6, x0_2_7, x0_2_8, x0_2_9, x0_3_0, x0_3_1,
       x0_3_2, x0_3_3, x0_3_4, x0_3_5, x0_3_6, x0_3_7, x0_3_8, x0_3_9,
       x1_0_0, x1_0_1, x1_0_2, x1_0_3, x1_0_4, x1_0_5, x1_0_6, x1_0_7,
       x1_0_8, x1_0_9, x1_1_0, x1_1_1, x1_1_2, x1_1_3, x1_1_4, x1_1_5,
       x1_1_6, x1_1_7, x1_1_8, x1_1_9, x1_2_0, x1_2_1, x1_2_2, x1_2_3,
       x1_2_4, x1_2_5, x1_2_6, x1_2_7, x1_2_8, x1_2_9, x1_3_0, x1_3_1,
       x1_3_2, x1_3_3, x1_3_4, x1_3_5, x1_3_6, x1_3_7, x1_3_8, x1_3_9,
       x2_0_0, x2_0_1, x2_0_2, x2_0_3, x2_0_4, x2_0_5, x2_0_6, x2_0_7,
       x2_0_8, x2_0_9, x2_1_0, x2_1_1, x2_1_2, x2_1_3, x2_1_4, x2_1_5,
       x2_1_6, x2_1_7, x2_1_8, x2_1_9, x2_2_0, x2_2_1, x2_2_2, x2_2_3,
       x2_2_4, x2_2_5, x2_2_6, x2_2_7, x2_2_8, x2_2_9, x2_3_0, x2_3_1,
      

In [249]:
limit = int(procedure[procedure["product_id"] == product[3]]["limit"])

In [250]:
limit

3

In [ ]:
    #约束条件0.2 每一个shift的每一个project并行数量不超过工位上限
    for j in range(s_num):
        for k in range(n_num):
            a = np.zeros([m_num, s_num, n_num])
            for i in range(m_num):
                a[i,j,k] = 1
            #找出该product所属的project的工位上限
            limit = int(procedure[procedure["product_id"] == product[k]]["limit"])
            constraint_k = sum(a.flatten()*variables) <= limit
            constraints.append(constraint_k)

In [72]:
team

,team_id,member,product_id,project_id,output
0,200001,"[100007, 100017, 100018]","[500020, 500021, 500022]",400010,1200


In [56]:
worker_new

['100002',
 '100003',
 '100004',
 '100005',
 '100006',
 '100007',
 '100008',
 '100009',
 '100010',
 '100011',
 '100012',
 '100013',
 '100014',
 '100015',
 '100016',
 '100017',
 '100018',
 '100019',
 '100020',
 '100021',
 '100022',
 '100024',
 '100025']

In [57]:
worker_new[5] in team_mem

True

In [58]:
team_pro

['500008', '500009', '500010']

In [59]:
product.index(team_pro[0])

2

In [60]:
team_mem

['100004', '100007', '100011']

In [148]:
product

['500006',
 '500007',
 '500008',
 '500009',
 '500010',
 '500011',
 '500012',
 '500013']

In [149]:
order

,order_id,project_id,project_name,product_id,product_name,delivery_quantity,delivery_date,total_output,quantity,days,create_date,change_date,checked
0,800065,400004,舍弗勒,500006,舍弗勒塑料球子,16000,2019-09-22,0,12800,13,2019-09-05 17:53:14,2019-09-05 17:53:14,YES
1,800059,400005,安全阀,500010,检查,24000,2020-02-04,3725,16220,148,2019-09-05 17:09:36,2019-09-05 18:07:50,NO
2,800058,400006,金属片,500013,手动成形,39000,2019-12-16,0,31200,98,2019-08-20 18:53:54,2019-09-06 10:47:07,NO
3,800065,400004,舍弗勒,500007,舍弗勒金属球子,16000,2019-09-22,2000,11200,13,2019-09-05 17:53:14,2019-09-05 17:53:14,YES
4,800058,400006,金属片,500011,剪切金属片,39000,2019-12-16,5000,27200,98,2019-08-20 18:53:54,2019-09-06 10:47:07,NO
5,800059,400005,安全阀,500008,预放组件,24000,2020-02-04,4125,15900,148,2019-09-05 17:09:36,2019-09-05 18:07:50,NO
6,800058,400006,金属片,500012,压机成形,39000,2019-12-16,4887,27290,98,2019-08-20 18:53:54,2019-09-06 10:47:07,NO
7,800059,400005,安全阀,500009,推安全阀,24000,2020-02-04,4225,15820,148,2019-09-05 17:09:36,2019-09-05 18:07:50,NO


In [173]:
order['product_id'][order['project_id']=='400005']

1    500010
5    500008
7    500009
Name: product_id, dtype: object

In [156]:
eff

,worker_id,project_id,product_id,date,output
0,100002,400003,500004,0000-00-00,400
1,100002,400003,500005,0000-00-00,74.25
2,100002,400004,500006,0000-00-00,508.27
3,100002,400004,500007,0000-00-00,375.68
4,100002,400006,500012,0000-00-00,324.59
...,...,...,...,...,...
358,100025,400012,500024,0000-00-00,180
359,100025,400013,500026,0000-00-00,140.1
360,100025,400017,500036,0000-00-00,400
361,100025,400007,500037,0000-00-00,400


In [158]:
eff['worker_id'][eff['product_id']=='500008']

23     100003
40     100004
54     100005
75     100007
123    100010
150    100011
170    100013
188    100014
207    100015
253    100018
274    100019
299    100020
325    100022
Name: worker_id, dtype: object

In [159]:
eff['worker_id'][eff['product_id']=='500009']

41     100004
76     100007
189    100014
Name: worker_id, dtype: object

In [160]:
eff['worker_id'][eff['product_id']=='500010']

151    100011
190    100014
208    100015
300    100020
Name: worker_id, dtype: object

In [ ]:
100004,100007,100011

In [174]:
team

,team_id,member,product_id,project_id,output
0,200001,"[100004, 100007, 100011]","[500008, 500009, 500010]",400005,1200


In [183]:
data = {"assignment": {"100002": ["500007", "500012", "500013", "500007"], "100003": ["500012", "500008", "500012", "500006"], "100004": ["500007", "500009", "500010", "500012"], "100005": ["500007", "500012", "500010", "500009"], "100006": ["500009", "500012", "500007", "500011"], "100007": ["500013", "500011", "500009", "500012"], "100008": ["500013", "500013", "500008", "500013"], "100009": ["500008", "500012", "500012", "500009"], "100010": ["500010", "500010", "500013", "500009"], "100011": ["500013", "500009", "500009", "500008"], "100012": ["500013", "500012", "500013", "500008"], "100013": ["500006", "500009", "500011", "500010"], "100014": ["500008", "500007", "500010", "500007"], "100015": ["500009", "500011", "500007", "500009"], "100016": ["500010", "500007", "500009", "500013"], "100017": ["500008", "500008", "500008", "500008"], "100018": ["500007", "500006", "500009", "500010"], "100019": ["500007", "500009", "500010", "500006"], "100020": ["500009", "500009", "500009", "500009"], "100021": ["500010", "500013", "500008", "500007"], "100022": ["500011", "500006", "500008", "500012"], "100024": ["500010", "500006", "500006", "500010"], "100025": ["500010", "500010", "500010", "500010"]}, "estimated output": {"500006": 6223.66, "500007": 10604.519999999999, "500008": 3773.18, "500009": 4894.4400000000005, "500010": 3827.16, "500011": 3600.0, "500012": 6756.18, "500013": 1262.96}, "total shifts": 92.0, "working load": 100.0, "estimated duration": {"800065": 3, "800059": 5, "800058": 25}}


In [184]:
data

{'assignment': {'100002': ['500007', '500012', '500013', '500007'],
  '100003': ['500012', '500008', '500012', '500006'],
  '100004': ['500007', '500009', '500010', '500012'],
  '100005': ['500007', '500012', '500010', '500009'],
  '100006': ['500009', '500012', '500007', '500011'],
  '100007': ['500013', '500011', '500009', '500012'],
  '100008': ['500013', '500013', '500008', '500013'],
  '100009': ['500008', '500012', '500012', '500009'],
  '100010': ['500010', '500010', '500013', '500009'],
  '100011': ['500013', '500009', '500009', '500008'],
  '100012': ['500013', '500012', '500013', '500008'],
  '100013': ['500006', '500009', '500011', '500010'],
  '100014': ['500008', '500007', '500010', '500007'],
  '100015': ['500009', '500011', '500007', '500009'],
  '100016': ['500010', '500007', '500009', '500013'],
  '100017': ['500008', '500008', '500008', '500008'],
  '100018': ['500007', '500006', '500009', '500010'],
  '100019': ['500007', '500009', '500010', '500006'],
  '100020': ['

In [187]:
i_all = []
k_all = []
for i in range(0, len(team_mem)):
    i_index = worker_new.index(team_mem[i])
    k_index = product.index(team_pro[i])
    i_all.append(i_index)
    k_all.append(k_index)

In [188]:
i_all

[2, 5, 9]

In [189]:
k_all

[2, 3, 4]

In [191]:
product

['500006',
 '500007',
 '500008',
 '500009',
 '500010',
 '500011',
 '500012',
 '500013']

In [192]:
{"assignment": {"100002": ["500007", "500012", "500013", "500007"], "100003": ["500012", "500008", "500012", "500006"], "100004": ["500007", "500009", "500010", "500012"], "100005": ["500007", "500012", "500010", "500009"], "100006": ["500009", "500012", "500007", "500011"], "100007": ["500013", "500011", "500009", "500012"], "100008": ["500013", "500013", "500008", "500013"], "100009": ["500008", "500012", "500012", "500009"], "100010": ["500010", "500010", "500013", "500009"], "100011": ["500013", "500009", "500009", "500008"], "100012": ["500013", "500012", "500013", "500008"], "100013": ["500006", "500009", "500011", "500010"], "100014": ["500008", "500007", "500010", "500007"], "100015": ["500009", "500011", "500007", "500009"], "100016": ["500010", "500007", "500009", "500013"], "100017": ["500008", "500008", "500008", "500008"], "100018": ["500007", "500006", "500009", "500010"], "100019": ["500007", "500009", "500010", "500006"], "100020": ["500009", "500009", "500009", "500009"], "100021": ["500010", "500013", "500008", "500007"], "100022": ["500011", "500006", "500008", "500012"], "100024": ["500010", "500006", "500006", "500010"], "100025": ["500010", "500010", "500010", "500010"]}, "estimated output": {"500006": 6223.66, "500007": 10604.519999999999, "500008": 3773.18, "500009": 4894.4400000000005, "500010": 3827.16, "500011": 3600.0, "500012": 6756.18, "500013": 1262.96}, "total shifts": 92.0, "working load": 100.0, "estimated duration": {"800065": 3, "800059": 5, "800058": 25}}

{'assignment': {'100002': ['500007', '500012', '500013', '500007'],
  '100003': ['500012', '500008', '500012', '500006'],
  '100004': ['500007', '500009', '500010', '500012'],
  '100005': ['500007', '500012', '500010', '500009'],
  '100006': ['500009', '500012', '500007', '500011'],
  '100007': ['500013', '500011', '500009', '500012'],
  '100008': ['500013', '500013', '500008', '500013'],
  '100009': ['500008', '500012', '500012', '500009'],
  '100010': ['500010', '500010', '500013', '500009'],
  '100011': ['500013', '500009', '500009', '500008'],
  '100012': ['500013', '500012', '500013', '500008'],
  '100013': ['500006', '500009', '500011', '500010'],
  '100014': ['500008', '500007', '500010', '500007'],
  '100015': ['500009', '500011', '500007', '500009'],
  '100016': ['500010', '500007', '500009', '500013'],
  '100017': ['500008', '500008', '500008', '500008'],
  '100018': ['500007', '500006', '500009', '500010'],
  '100019': ['500007', '500009', '500010', '500006'],
  '100020': ['

In [193]:
{"assignment": {"100002": ["500007", "500012", "500013", "500007"], "100003": ["500012", "500008", "500012", "500006"], "100004": ["500007", "500009", "500010", "500012"], "100005": ["500007", "500012", "500010", "500009"], "100006": ["500009", "500012", "500007", "500011"], "100007": ["500013", "500011", "500009", "500012"], "100008": ["500013", "500013", "500008", "500013"], "100009": ["500008", "500012", "500012", "500009"], "100010": ["500010", "500010", "500013", "500009"], "100011": ["500013", "500009", "500009", "500008"], "100012": ["500013", "500012", "500013", "500008"], "100013": ["500006", "500009", "500011", "500010"], "100014": ["500008", "500007", "500010", "500007"], "100015": ["500009", "500011", "500007", "500009"], "100016": ["500010", "500007", "500009", "500013"], "100017": ["500008", "500008", "500008", "500008"], "100018": ["500007", "500006", "500009", "500010"], "100019": ["500007", "500009", "500010", "500006"], "100020": ["500009", "500009", "500009", "500009"], "100021": ["500010", "500013", "500008", "500007"], "100022": ["500011", "500006", "500008", "500012"], "100024": ["500010", "500006", "500006", "500010"], "100025": ["500010", "500010", "500010", "500010"]}, "estimated output": {"500006": 6223.66, "500007": 10604.519999999999, "500008": 3773.18, "500009": 4894.4400000000005, "500010": 3827.16, "500011": 3600.0, "500012": 6756.18, "500013": 1262.96}, "total shifts": 92.0, "working load": 100.0, "estimated duration": {"800065": 3, "800059": 5, "800058": 25}}

{'assignment': {'100002': ['500007', '500012', '500013', '500007'],
  '100003': ['500012', '500008', '500012', '500006'],
  '100004': ['500007', '500009', '500010', '500012'],
  '100005': ['500007', '500012', '500010', '500009'],
  '100006': ['500009', '500012', '500007', '500011'],
  '100007': ['500013', '500011', '500009', '500012'],
  '100008': ['500013', '500013', '500008', '500013'],
  '100009': ['500008', '500012', '500012', '500009'],
  '100010': ['500010', '500010', '500013', '500009'],
  '100011': ['500013', '500009', '500009', '500008'],
  '100012': ['500013', '500012', '500013', '500008'],
  '100013': ['500006', '500009', '500011', '500010'],
  '100014': ['500008', '500007', '500010', '500007'],
  '100015': ['500009', '500011', '500007', '500009'],
  '100016': ['500010', '500007', '500009', '500013'],
  '100017': ['500008', '500008', '500008', '500008'],
  '100018': ['500007', '500006', '500009', '500010'],
  '100019': ['500007', '500009', '500010', '500006'],
  '100020': ['

In [ ]:
100017 100020 100025

In [1]:
import numpy as np

In [62]:
np.array(worker)

array(['100002', '100003', '100004', '100005', '100006', '100007',
       '100008', '100009', '100010', '100011', '100012', '100013',
       '100014', '100015', '100016', '100017', '100018', '100019',
       '100020', '100021', '100022', '100023', '100024', '100025'],
      dtype='<U6')

In [65]:
index = np.argwhere(np.array(worker) == worker[12seq = np.arange(index*s_num*n_num,(index+1)*s_num*n_num)]).tolist()[0][0]

In [66]:
index

12

In [67]:
seq = np.arange(index*s_num*n_num,(index+1)*s_num*n_num)

In [68]:
seq

array([384, 385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 396,
       397, 398, 399, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409,
       410, 411, 412, 413, 414, 415])

In [70]:
work_efficiency_new

NameError: name 'work_efficiency_new' is not defined

In [71]:
worker_efficiency_new

array([[ 508.27,  375.68,    0.  ,    0.  ,    0.  ,    0.  ,  324.59,
           0.  ],
       [ 300.  ,  411.12,  200.  ,    0.  ,    0.  ,    0.  ,  189.6 ,
         184.71],
       [ 407.48,  598.87,  300.  ,  418.87,    0.  ,    0.  ,    0.  ,
           0.  ],
       [ 464.11,  400.  ,  300.  ,    0.  ,    0.  , 1600.  ,  400.  ,
         256.  ],
       [ 400.  ,  400.  ,    0.  ,    0.  ,    0.  ,    0.  ,    0.  ,
           0.  ],
       [ 426.  ,  254.64,  200.  ,  508.8 ,    0.  ,    0.  ,  305.94,
         293.48],
       [ 364.13,  342.65,    0.  ,    0.  ,    0.  , 1600.  ,  342.  ,
         104.53],
       [ 460.62,  420.08,    0.  ,    0.  ,    0.  ,    0.  ,  340.95,
           0.  ],
       [ 500.  ,  541.29,  247.5 ,    0.  ,    0.  , 1800.  ,  600.  ,
         645.25],
       [ 500.  ,  624.15,  250.  ,    0.  ,  398.87,    0.  ,  600.  ,
         283.63],
       [ 487.1 ,  467.67,    0.  ,    0.  ,    0.  ,    0.  ,    0.  ,
           0.  ],
       [ 420.78,  382

In [73]:
efficiency

,0,1,2,3,4
0,100002,400003,500004,0000-00-00,400
1,100002,400003,500005,0000-00-00,74.25
2,100002,400004,500006,0000-00-00,508.27
3,100002,400004,500007,0000-00-00,375.68
4,100002,400006,500012,0000-00-00,324.59
...,...,...,...,...,...
358,100025,400012,500024,0000-00-00,180
359,100025,400013,500026,0000-00-00,140.1
360,100025,400017,500036,0000-00-00,400
361,100025,400007,500037,0000-00-00,400


In [76]:
worker[9]

'100011'

In [77]:
{"assignment": {"100002": ["500006", "500006", "500006", "500006"], "100003": ["500007", "500006", "500006", "500006"], "100004": ["500012", "500012", "500012", "500012"], "100005": ["500006", "500006", "500006", "500006"], "100006": ["500011", "500011", "500011", "500011"], "100007": ["500006", "500012", "500012", "500012"], "100008": ["500012", "500012", "500012", "500012"], "100009": ["500006", "500006", "500006", "500006"], "100010": ["500011", "500011", "500011", "500011"], "100011": ["500008", "500007", "500007", "500007"], "100012": ["500007", "500007", "500007", "500007"], "100013": ["500007", "500007", "500007", "500007"], "100014": ["500013", "500013", "500013", "500013"], "100015": ["500007", "500007", "500007", "500007"], "100016": ["500012", "500012", "500012", "500012"], "100017": ["500008", "500008", "500008", "500008"], "100018": ["500011", "500011", "500011", "500011"], "100019": ["500007", "500007", "500007", "500007"], "100020": ["500009", "500009", "500009", "500009"], "100021": ["500011", "500011", "500011", "500011"], "100022": ["500006", "500006", "500006", "500006"], "100024": ["500011", "500011", "500011", "500011"], "100025": ["500010", "500010", "500010", "500010"]}, "estimated output": {"500006": 21311.420000000002, "500007": 18092.76, "500008": 2973.18, "500009": 4070.4, "500010": 3190.96, "500011": 68800.0, "500012": 16009.419999999998, "500013": 7140.0}, "total shifts": 92.0, "working load": 100.0, "estimated duration": {"800065": 1, "800059": 6, "800058": 5}}

{'assignment': {'100002': ['500006', '500006', '500006', '500006'],
  '100003': ['500007', '500006', '500006', '500006'],
  '100004': ['500012', '500012', '500012', '500012'],
  '100005': ['500006', '500006', '500006', '500006'],
  '100006': ['500011', '500011', '500011', '500011'],
  '100007': ['500006', '500012', '500012', '500012'],
  '100008': ['500012', '500012', '500012', '500012'],
  '100009': ['500006', '500006', '500006', '500006'],
  '100010': ['500011', '500011', '500011', '500011'],
  '100011': ['500008', '500007', '500007', '500007'],
  '100012': ['500007', '500007', '500007', '500007'],
  '100013': ['500007', '500007', '500007', '500007'],
  '100014': ['500013', '500013', '500013', '500013'],
  '100015': ['500007', '500007', '500007', '500007'],
  '100016': ['500012', '500012', '500012', '500012'],
  '100017': ['500008', '500008', '500008', '500008'],
  '100018': ['500011', '500011', '500011', '500011'],
  '100019': ['500007', '500007', '500007', '500007'],
  '100020': ['

In [78]:
worker

['100002',
 '100003',
 '100004',
 '100005',
 '100006',
 '100007',
 '100008',
 '100009',
 '100010',
 '100011',
 '100012',
 '100013',
 '100014',
 '100015',
 '100016',
 '100017',
 '100018',
 '100019',
 '100020',
 '100021',
 '100022',
 '100023',
 '100024',
 '100025']

In [90]:
eff[eff['worker_id']=='100004']

,worker_id,project_id,product_id,date,output
36,100004,400002,500002,0000-00-00,935.9
37,100004,400003,500005,0000-00-00,80.45
38,100004,400004,500006,0000-00-00,407.48
39,100004,400004,500007,0000-00-00,598.87
40,100004,400005,500008,0000-00-00,300
41,100004,400005,500009,0000-00-00,418.87
42,100004,400007,500016,0000-00-00,250
43,100004,400008,500017,0000-00-00,256.69
44,100004,400012,500025,0000-00-00,144
45,100004,400014,500028,0000-00-00,300


In [85]:
worker_efficiency_new

array([[ 508.27,  375.68,    0.  ,    0.  ,    0.  ,    0.  ,  324.59,
           0.  ],
       [ 300.  ,  411.12,  200.  ,    0.  ,    0.  ,    0.  ,  189.6 ,
         184.71],
       [ 407.48,  598.87,  300.  ,  418.87,    0.  ,    0.  ,    0.  ,
           0.  ],
       [ 464.11,  400.  ,  300.  ,    0.  ,    0.  , 1600.  ,  400.  ,
         256.  ],
       [ 400.  ,  400.  ,    0.  ,    0.  ,    0.  ,    0.  ,    0.  ,
           0.  ],
       [ 426.  ,  254.64,  200.  ,  508.8 ,    0.  ,    0.  ,  305.94,
         293.48],
       [ 364.13,  342.65,    0.  ,    0.  ,    0.  , 1600.  ,  342.  ,
         104.53],
       [ 460.62,  420.08,    0.  ,    0.  ,    0.  ,    0.  ,  340.95,
           0.  ],
       [ 500.  ,  541.29,  247.5 ,    0.  ,    0.  , 1800.  ,  600.  ,
         645.25],
       [ 500.  ,  624.15,  250.  ,    0.  ,  398.87,    0.  ,  600.  ,
         283.63],
       [ 487.1 ,  467.67,    0.  ,    0.  ,    0.  ,    0.  ,    0.  ,
           0.  ],
       [ 420.78,  382

In [91]:
{"assignment": {"100002": ["500006", "500006", "500006", "500006"], "100003": ["500007", "500006", "500006", "500006"], "100004": ["500012", "500012", "500012", "500012"], "100005": ["500006", "500006", "500006", "500006"], "100006": ["500011", "500011", "500011", "500011"], "100007": ["500006", "500012", "500012", "500012"], "100008": ["500012", "500012", "500012", "500012"], "100009": ["500006", "500006", "500006", "500006"], "100010": ["500011", "500011", "500011", "500011"], "100011": ["500008", "500007", "500007", "500007"], "100012": ["500007", "500007", "500007", "500007"], "100013": ["500007", "500007", "500007", "500007"], "100014": ["500013", "500013", "500013", "500013"], "100015": ["500007", "500007", "500007", "500007"], "100016": ["500012", "500012", "500012", "500012"], "100017": ["500008", "500008", "500008", "500008"], "100018": ["500011", "500011", "500011", "500011"], "100019": ["500007", "500007", "500007", "500007"], "100020": ["500009", "500009", "500009", "500009"], "100021": ["500011", "500011", "500011", "500011"], "100022": ["500006", "500006", "500006", "500006"], "100024": ["500011", "500011", "500011", "500011"], "100025": ["500010", "500010", "500010", "500010"]}, "estimated output": {"500006": 21311.420000000002, "500007": 18092.76, "500008": 2973.18, "500009": 4070.4, "500010": 3190.96, "500011": 68800.0, "500012": 16009.419999999998, "500013": 7140.0}, "total shifts": 92.0, "working load": 100.0, "estimated duration": {"800065": 1, "800059": 6, "800058": 5}}

{'assignment': {'100002': ['500006', '500006', '500006', '500006'],
  '100003': ['500007', '500006', '500006', '500006'],
  '100004': ['500012', '500012', '500012', '500012'],
  '100005': ['500006', '500006', '500006', '500006'],
  '100006': ['500011', '500011', '500011', '500011'],
  '100007': ['500006', '500012', '500012', '500012'],
  '100008': ['500012', '500012', '500012', '500012'],
  '100009': ['500006', '500006', '500006', '500006'],
  '100010': ['500011', '500011', '500011', '500011'],
  '100011': ['500008', '500007', '500007', '500007'],
  '100012': ['500007', '500007', '500007', '500007'],
  '100013': ['500007', '500007', '500007', '500007'],
  '100014': ['500013', '500013', '500013', '500013'],
  '100015': ['500007', '500007', '500007', '500007'],
  '100016': ['500012', '500012', '500012', '500012'],
  '100017': ['500008', '500008', '500008', '500008'],
  '100018': ['500011', '500011', '500011', '500011'],
  '100019': ['500007', '500007', '500007', '500007'],
  '100020': ['

In [94]:
worker_new

['100002',
 '100003',
 '100004',
 '100005',
 '100006',
 '100007',
 '100008',
 '100009',
 '100010',
 '100011',
 '100012',
 '100013',
 '100014',
 '100015',
 '100016',
 '100017',
 '100018',
 '100019',
 '100020',
 '100021',
 '100022',
 '100024',
 '100025']

In [96]:
len(worker_efficiency_new)

23

In [97]:
team

,team_id,member,product_id,project_id,output
0,200001,"[100004, 100007, 100011]","[500008, 500009, 500010]",400005,1200


In [98]:
product

['500006',
 '500007',
 '500008',
 '500009',
 '500010',
 '500011',
 '500012',
 '500013']

In [119]:
for index in team.index:
    item = team.loc[index]
    mem = item['member']
    print(mem)

100004,100007,100011


In [121]:
team_worker_eff = worker_efficiency_new
for index in team.index:
    item = team.loc[index]
    mem = item['member']
    pro = item['product_id']
    eff_new = item['output']
    for i in range(len(mem)):
        mem_index = worker_new.index(mem[i])
        pro_index = product.index(pro[i])
        team_worker_eff[mem_index][pro_index] = eff_new

In [ ]:
def eff_transform(worker_efficiency, worker, product):
    